### 모듈 불러오기

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import pickle
from konlpy.tag import Mecab
mecab = Mecab()

### 파일 불러오기 및 결측치 처리

In [2]:
data = pd.read_csv("./open/train.csv")
data = data.dropna()

### POS 태깅

In [3]:
%%time
def pos(x):
    try:
        text = ''
        for word, pos in mecab.pos(str(x)):
            if pos[0] in ["N","V","M"]:
                if type(re.search("\W+|[A-z0-9]", word))!=re.Match and len(word)!=1:
                    text+=" "+word
        return text.strip()
    
    except:
        pass
    
data["tagged_goal"] = data["요약문_연구목표"].apply(pos)
data["tagged_cont"] = data["요약문_연구내용"].apply(pos)
data["tagged_effect"] = data["요약문_기대효과"].apply(pos)
data["요약문_한글키워드"] = data["요약문_한글키워드"].str.replace(" ", "")

CPU times: user 4min 24s, sys: 601 ms, total: 4min 25s
Wall time: 4min 23s


### Tfidf Vector 만들기

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect1 = TfidfVectorizer(min_df = 0.01, max_df = 0.9)
tfidf_goal = tfidf_vect1.fit_transform(data["tagged_goal"])

In [5]:
tfidf_vect2 = TfidfVectorizer(min_df = 0.01, max_df = 0.9)
tfidf_cont = tfidf_vect2.fit_transform(data["tagged_cont"])

In [6]:
tfidf_vect3 = TfidfVectorizer(min_df = 0.01, max_df = 0.9)
tfidf_effect = tfidf_vect3.fit_transform(data["tagged_effect"])

In [29]:
tfidf_vect4 = TfidfVectorizer(min_df = 0.0005)
tfidf_keywords = tfidf_vect4.fit_transform(data["요약문_한글키워드"])

### DataFrame 만들기 (필요 시 살리기)

In [30]:
goal_df = pd.DataFrame(tfidf_goal.todense(), columns = tfidf_vect1.get_feature_names())
cont_df = pd.DataFrame(tfidf_cont.todense(), columns = tfidf_vect2.get_feature_names())
effect_df = pd.DataFrame(tfidf_effect.todense(), columns = tfidf_vect3.get_feature_names())
keyword_df = pd.DataFrame(tfidf_keywords.todense(), columns = tfidf_vect4.get_feature_names())

In [9]:
with open("goal_df.pkl", "wb") as f:
    pickle.dump(goal_df, f)

In [10]:
with open("cont_df.pkl", "wb") as f:
    pickle.dump(cont_df, f)

In [11]:
with open("effect_df.pkl", "wb") as f:
    pickle.dump(effect_df, f)

In [32]:
with open("keyword_df.pkl", "wb") as f:
    pickle.dump(keyword_df, f)

In [13]:
goal_df.shape

(171138, 745)

In [14]:
effect_df.shape

(171138, 927)

In [15]:
cont_df.shape

(171138, 1121)

In [31]:
keyword_df.shape

(171138, 896)

In [17]:
data.shape

(171138, 16)